# <center> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 3. Визуальный анализ данных и построение признаков

На 3 неделе мы займемся визуальным анализом данных и построением признаков. Сначала мы вместе построим и проанализируем несколько признаков, потом Вы сможете сами придумать и описать различные признаки. Задание имеет вид Peer-Review, так что творчество здесь активно приветствуется. Если задействуете IPython-виджеты, библиотеку Plotly, анимации и прочий интерактив, всем от этого будет только лучше.

**План 3 недели:**
 - Часть 1. Построение признаков и визуальный анализ данных
 - Часть 2. Визуальный анализ данных
 - Часть 3. Дальнейшее построение признаков
 

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Поиск структуры в данных":**
   - [Задача визуализации](https://www.coursera.org/learn/unsupervised-learning/lecture/hlvlT/zadacha-vizualizatsii)
   - [Визуализация данных в sklearn](https://www.coursera.org/learn/unsupervised-learning/lecture/ityMo/vizualizatsiia-dannykh-v-sklearn)
   
**Также в задании будет использоваться библиотека Seaborn (ее можно дополнительно установить командой *conda install seaborn*), будет полезно обращаться к документациям [Matplotlib](http://matplotlib.org/users/) и [Seaborn](http://seaborn.pydata.org/), а также к примерам визуализации, описанным на StackOverflow.**



## Часть 1. Построение признаков

In [2]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import numpy as np
import pandas as pd
pd.set_option('display.max.columns', 25)
import pickle
#conda install seaborn
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Создайте на основе функций *prepare_train_set* и *prepare_sparse_train_set_window*  новую – *prepare_train_set_with_fe*, (от "feature engineering"), создайте следующие 13 признаков:**
- time_diff1, ..., time_diff9 – промежутки между посещением 1 и 2 сайта, ..., 9 и 10 сайта в сессии (в секундах). Посещения сайтов не сторого упорядочены по времени, поэтому эти величины где-то могут быть отрицательными. Если сайтов в сессии меньше 10, разницы можно сделать нулевыми
- session_timespan – продолжительность сессии. Из-за того, что посещения сайтов не сторого упорядочены по времени, продолжительностью сессии будем считать разницу между максимальным и минимальным временем посещения сайтов в сессии (в секундах).
- #unique_sites – число уникальных сайтов в сессии 
- start_hour – час начала сессии (то есть час в записи минимального timestamp среди десяти)
- day_of_week – день недели (то есть день недели в записи минимального timestamp среди десяти)

Функция должна возвращать новый DataFrame (как возвращала функция *prepare_train_set*), только признаков должно быть на 13 больше. Порядок, в котором добавляются признаки: *site1*, ... *site10*, *time_diff1*, ..., *time_diff9*, *session_timespan*, *#unique_sites*, *start_hour*,*day_of_week*,*target* (это видно и чуть ниже по тому, как функция вызывается).

In [3]:
def construct_item(sites, user_id, session_length, timestamps):
    if len(sites) == 0:
        return None
    
    item = {'target': user_id}
    
    if len(sites) != session_length:
        sites = sites + [0]*(session_length - len(sites))
        timestamps = timestamps + [timestamps[-1]]*(session_length - len(timestamps))
    for i in range(session_length):
        item['site' + str(i+1)] = sites[i]
        if i < session_length - 1:
            item['time_diff' + str(i+1)] = timestamps[i+1] - timestamps[i]
            
    item['session_timespan'] = max(timestamps) - min(timestamps)
    item['#unique_sites'] = len(list(set(filter(lambda x: x != 0, sites))))
    item['start_hour'] = datetime.datetime.fromtimestamp(min(timestamps)).hour
    item['day_of_week'] = datetime.datetime.fromtimestamp(min(timestamps)).weekday()
    return item

def prepare_train_set_with_fe(csv_files_mask, site_freq_path, feature_names,
                                    session_length=10, window_size=10):
    pkl_file = open(site_freq_path, 'rb')
    site_freq = pickle.load(pkl_file)
    
    sessions = []
    
    for filename in glob(csv_files_mask):
        with open(filename) as input_file:
            data = filter(lambda x: x!= '', map(lambda x: x.strip(), input_file.readlines()))

            user_id = int(data[0].split(',')[0])
            n = len(data)
            for i in range(0, n, window_size):
                session = map(lambda x: x.split(',')[2], data[i:i + session_length])
                times = map(lambda x: x.split(',')[1], data[i:i + session_length])
                if len(session) == 0:
                    continue
                session_ids = map(lambda x: site_freq[x][0], session)
                timestamps = map(lambda x: int(datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S').strftime('%s')), 
                                 times)
                
                sessions.append(construct_item(session_ids, user_id, session_length, timestamps))
    
    df = pd.DataFrame(sessions)
    df = df.drop_duplicates()

    columns = map(lambda x: 'site' + str(x+1), range(session_length)) + ['target']
    return df[feature_names]

**Проверим функцию на игрушечном примере.**

In [4]:
feature_names = ['site' + str(i) for i in range(1,11)] + \
                ['time_diff' + str(j) for j in range(1,10)] + \
                ['session_timespan', '#unique_sites', 'start_hour', 'day_of_week', 'target']
train_data_toy  = prepare_train_set_with_fe('capstone_websites_data/3users_toy/*', 
                                   site_freq_path='capstone_websites_data/site_freq_3users.pkl',
                                   feature_names=feature_names, session_length=10)

In [5]:
train_data_toy

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,time_diff1,time_diff2,time_diff3,time_diff4,time_diff5,time_diff6,time_diff7,time_diff8,time_diff9,session_timespan,#unique_sites,start_hour,day_of_week,target
0,1,2,2,3,2,4,5,6,7,8,287,1184,6278,186,2,1,2,3,55,7998,8,9,4,1
1,1,4,4,4,0,0,0,0,0,0,2,3,55,0,0,0,0,0,0,60,2,12,4,1
2,1,2,9,9,2,0,0,0,0,0,287,1184,6278,186,0,0,0,0,0,7935,3,9,4,2
3,10,4,2,4,2,4,4,6,11,10,287,1184,6278,186,2,1,2,3,55,7998,5,9,4,3
4,10,4,2,0,0,0,0,0,0,0,287,1184,0,0,0,0,0,0,0,1471,3,12,4,3


**Примените функцию *prepare_train_set_with_fe* к данным по 10 пользователям, укажите session_length=10.**

In [6]:
train_data_10users = prepare_train_set_with_fe('capstone_websites_data/10users/*', 
                                   site_freq_path='capstone_websites_data/site_freq_10users.pkl',
                                   feature_names=feature_names, session_length=10)

In [7]:
train_data_10users.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,time_diff1,time_diff2,time_diff3,time_diff4,time_diff5,time_diff6,time_diff7,time_diff8,time_diff9,session_timespan,#unique_sites,start_hour,day_of_week,target
0,1,2,2,3,4,5,6,7,8,7,273,3,28,916,-1523,10,0,0,1,1523,8,8,4,31
1,7,8,7,7,9,10,11,12,13,14,1,0,1,0,0,1,0,163,105,271,8,8,4,31
2,7,15,16,17,17,8,17,17,10,3,4,3,8,1,0,14,1,242,0,273,7,8,4,31
3,18,19,20,17,21,17,17,17,17,22,1,0,0,2,1,1,25,1,0,31,6,8,4,31
4,23,24,25,17,26,27,28,29,30,31,0,0,1,1,1,1,0,1,1,6,10,8,4,31


**Примените функцию *prepare_train_set_with_fe* к данным по 150 пользователям, укажите session_length=10.**

In [8]:
train_data_150users = prepare_train_set_with_fe('capstone_websites_data/150users/*', 
                                   site_freq_path='capstone_websites_data/site_freq_150users.pkl',
                                   feature_names=feature_names, session_length=10)

**Сохраните в pickle-файлы *time_diff1*,..., *time_diff9, session_timespan, #unique_sites, start_hour* и *day_of_week* для 10 и 150 пользователей.**

In [9]:
new_feature_names = ['time_diff' + str(j) for j in range(1,10)] + \
                ['session_timespan', '#unique_sites', 'start_hour', 'day_of_week']

new_features_10users = train_data_10users[new_feature_names]
new_features_150users = train_data_150users[new_feature_names]

In [10]:
with open('new_features_10users.pkl', 'wb') as new_features_10users_pkl:
    pickle.dump(new_features_10users, new_features_10users_pkl)
with open('new_features_150users.pkl', 'wb') as new_features_150users_pkl:
    pickle.dump(new_features_150users, new_features_150users_pkl)

## Часть 2. Визуальный анализ данных

**Забавы ради, потехи для дадим пользователям имена и ассоциируем с ними цвета.**

In [11]:
id_name_dict = {128: 'Mary-Kate', 39: 'Ashley', 207: 'Lindsey', 127: 'Naomi', 237: 'Avril',
               33: 'Bob', 50: 'Bill', 31: 'John', 100: 'Dick', 241: 'Ed'}
train_data_10users['target'] = train_data_10users['target'].map(id_name_dict)

In [12]:
color_dic = {'Mary-Kate': 'pink', 'Ashley': 'darkviolet', 'Lindsey':'blueviolet', 
             'Naomi': 'hotpink', 'Avril': 'orchid', 
             'Bob': 'firebrick', 'Bill': 'gold', 'John': 'forestgreen', 
             'Dick': 'slategrey', 'Ed':'brown'}

**1. Постройте гистограмму распределения длины сессии в секундах (*session_timespan*). Ограничьте по *x* значением 200 (иначе слишком тяжелый хвост). Сделайте гистограмму цвета *darkviolet*, подпишите оси по-русски.**

In [13]:
import plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go

init_notebook_mode(connected = True)

In [14]:
data = [go.Histogram(
    x = train_data_10users[train_data_10users.session_timespan <= 200]['session_timespan'],
    marker = {'color': 'darkviolet'}
)]
layout = go.Layout(
    xaxis=dict(
        title='Длина сессии, сек'
    ),
    yaxis=dict(
        title='Число сессий'
    )
)

fig = go.Figure(data = data, layout = layout)
iplot(fig, show_link=False)

**2. Постройте гистограмму распределения числа уникальных сайтов в сессии (*#unique_sites*). Сделайте гистограмму цвета *aqua*, подпишите оси по-русски.**

In [15]:
data = [go.Histogram(
    x = train_data_10users['#unique_sites'] ,
    marker = {'color': 'aqua'}
)]
layout = go.Layout(
    xaxis=dict(
        title='Число уникальных сайтов в сессии'
    ),
    yaxis=dict(
        title='Число сессий'
    )
)

fig = go.Figure(data = data, layout = layout)
iplot(fig, show_link=False)

**3. Постройте гистограммы распределения числа уникальных сайтов в сессии (*#unique_sites*) для каждого из 10 пользователей по отдельности. Используйте *subplots*, чтоб разместить все 10 картинок на одной большой. Пометьте легендой каждую картинку – на легенде должно быть написано имя пользователя. Для каждого пользователя раскрасьте гистограмму его/ее цветом (*color_dic*). Подпишите оси по-русски в каждой из 10 гистограмм.**

In [16]:
from plotly import tools

In [17]:
fig = tools.make_subplots(rows=5, cols=2)

for idx, (user, sub_df) in  enumerate(pd.groupby(train_data_10users, 'target')): 
    trace = go.Histogram(
        x = sub_df['#unique_sites'] ,
        marker = {'color': color_dic[user]},
        name = user
    )
    fig.append_trace(trace, int(idx/2) + 1, int(idx % 2) + 1)
    fig['layout']['xaxis' + str(idx + 1)].update(title = 'Число уникальных сайтов в сессии')
    fig['layout']['yaxis' + str(idx + 1)].update(title = 'Число сессий')
    
fig['layout'].update(height=1100, width=800)
iplot(fig, show_link = False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]  
[ (2,1) x3,y3 ]    [ (2,2) x4,y4 ]  
[ (3,1) x5,y5 ]    [ (3,2) x6,y6 ]  
[ (4,1) x7,y7 ]    [ (4,2) x8,y8 ]  
[ (5,1) x9,y9 ]    [ (5,2) x10,y10 ]



**4. Постройте гистограмму распределения часа начала сессии (*start_hour*). Сделайте гистограмму цвета *darkgreen*, подпишите оси по-русски.**

In [18]:
data = [go.Histogram(
    x = train_data_10users['start_hour'] ,
    marker = {'color': 'dark_green'}
)]
layout = go.Layout(
    xaxis=dict(
        title='Час начала сессии'
    ),
    yaxis=dict(
        title='Число сессий'
    )
)

fig = go.Figure(data = data, layout = layout)
iplot(fig, show_link=False)

**5. Постройте гистограммы распределения часа начала сессии (*start_hour*) для каждого из 10 пользователей по отдельности. Используйте *subplots*, чтоб разместить все 10 картинок на одной большой. Пометьте легендой каждую картинку – на легенде должно быть написано имя пользователя. Для каждого пользователя раскрасьте гистограмму его/ее цветом (*color_dic*). Подпишите оси по-русски в каждой из 10 гистограмм.**

In [19]:
fig = tools.make_subplots(rows=5, cols=2)

for idx, (user, sub_df) in  enumerate(pd.groupby(train_data_10users, 'target')): 
    trace = go.Histogram(
        x = sub_df['start_hour'] ,
        marker = {'color': color_dic[user]},
        name = user
    )
    fig.append_trace(trace, int(idx/2) + 1, int(idx % 2) + 1)
    fig['layout']['xaxis' + str(idx + 1)].update(title = 'Час начала сессии')
    fig['layout']['yaxis' + str(idx + 1)].update(title = 'Число сессий')
    
fig['layout'].update(height=1100, width=800)
iplot(fig, show_link = False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]  
[ (2,1) x3,y3 ]    [ (2,2) x4,y4 ]  
[ (3,1) x5,y5 ]    [ (3,2) x6,y6 ]  
[ (4,1) x7,y7 ]    [ (4,2) x8,y8 ]  
[ (5,1) x9,y9 ]    [ (5,2) x10,y10 ]



**6. Постройте гистограмму распределения дня недели, в который началась сессия (*day_of_week*). Сделайте гистограмму цвета *sienna*, подпишите оси по-русски.**

In [44]:
data = [go.Histogram(
    x = train_data_10users['day_of_week'] ,
    marker = {'color': 'sienna'}
)]
layout = go.Layout(
    xaxis=dict(
        title='День недели, в который началась сессия'
    ),
    yaxis=dict(
        title='Число сессий'
    )
)

fig = go.Figure(data = data, layout = layout)
iplot(fig, show_link=False)

**7. Постройте гистограммы распределения дня недели, в который началась сессия (*day_of_week*) для каждого из 10 пользователей по отдельности. Используйте *subplots*, чтоб разместить все 10 картинок на одной большой. Измените метки по оси *X* на ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс'] – метод *set_xticklabels*. Пометьте легендой каждую картинку – на легенде должно быть написано имя пользователя. Для каждого пользователя раскрасьте гистограмму его/ее цветом (*color_dic*). Подпишите по-русски название каждой из 10 гистограмм.**

In [21]:
fig = tools.make_subplots(rows=5, cols=2, subplot_titles=['Распределение дня недели']*10)

for idx, (user, sub_df) in  enumerate(pd.groupby(train_data_10users, 'target')): 
    trace = go.Histogram(
        x = sub_df['day_of_week'] ,
        marker = {'color': color_dic[user]},
        name = user
    )
    fig.append_trace(trace, int(idx/2) + 1, int(idx % 2) + 1)
    fig['layout']['yaxis' + str(idx + 1)].update(title = 'Число сессий')
    fig['layout']['xaxis' + str(idx+1)].update(ticktext = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс'])
    fig['layout']['xaxis' + str(idx+1)].update(tickvals = range(7))
    
fig['layout'].update(height=1100, width=800)
iplot(fig, show_link = False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]  
[ (2,1) x3,y3 ]    [ (2,2) x4,y4 ]  
[ (3,1) x5,y5 ]    [ (3,2) x6,y6 ]  
[ (4,1) x7,y7 ]    [ (4,2) x8,y8 ]  
[ (5,1) x9,y9 ]    [ (5,2) x10,y10 ]



**8. Сделайте выводы про каждого пользователя по построенным графикам.**

* **Ashley**: пик активности приходится с 8 до 11 утра, есть еще меньшая активность с 14 до 16, вечером активности совсем нет; активность только по рабочим дням с пиком в среду
* **Avril**: активность есть с 9 утра до 19 вечера, с пиками в 13-14 часов и 17 часов; активность наблюдается в течение всей недели (больше всего во вторник-четверг и воскресение)
* **Bill**: основная активность наблюдается с 8 до 9 утра и затем еще днем с 14 до 18 часов; активность наблюдается в течение всей недели, больше всего в первую ее половину (понедельник - среда)
* **Bob**: активность наблюдается с 8 до 17, пики приходятся на 10 и 15 часов; активность только по рабочим дням, в основном, во вторник и четверг-пятницу
* **Dick**: препочитает посещать одни и те же сайты (много сессий с 1-2 уникальными сайтами); активность есть только в среду, пятницу - воскресение (больше всего в субботу); 
* **Ed**: достаточно равномерная активность с 8 до 21 часов, но есть яркий пик в 16 часов; нет активности в пятницу, больше всего во вторник и выходные
* **John**: активность наблюдается с 8 до 17 часов, пик приходится на 15 часов; активность только по будням, больше всего в конце недели (среда - пятница)
* **Lindsey**: активность наблюдается с 8 до 17 часов, пики приходятся на 12 и 14 часов; активность есть в течение всей недели, больше по будням, в основном, во вторник и среду
* **Mary-Kate**: препочитает посещать одни и те же сайты (много сессий с 1-2 уникальными сайтами), активность есть с 9 до 22 часов, пики активности приходятся на утро в 9-10 часов и на вечер в 20 часов; активность есть в течение всей недели, больше по выходным
* **Naomi**: пик активности приходится на середину дня в 13-14 часов; активность есть в течение всей недели, больше по будням, пик во вторник

**Загрузите сохраненный ранее в pickle-файл частотный словарь сайтов для 10 пользователей. **

In [22]:
pkl_file = open('capstone_websites_data/site_freq_10users.pkl', 'rb')
site_freq = pickle.load(pkl_file)

In [23]:
site_freq_df = pd.DataFrame.from_dict(site_freq).T
site_freq_df.columns = ['site_id', 'site_freq']
site_freq_df.sort_values('site_freq', ascending = False, inplace = True)
site_freq_df.head(10)

,site_id,site_freq
s.youtube.com,209,8300
www.google.fr,51,7813
www.google.com,8,5441
mail.google.com,183,4158
www.facebook.com,3,4141
apis.google.com,129,3758
r3---sn-gxo5uxg-jqbe.googlevideo.com,700,3244
r1---sn-gxo5uxg-jqbe.googlevideo.com,213,3094
plus.google.com,188,2630
accounts.google.com,184,2089


**Определите топ-10 самых посещаемых сайтов (*top10_sites*) и соответствующие кол-ва посещений (*top10_freqs*).**

In [24]:
top10_freqs = site_freq_df.head(10).site_freq.values.tolist()
top10_sites = site_freq_df.head(10).index.values.tolist()

**9. Нарисуйте *seaborn barplot*, показывающий частоты посещений топ-10 сайтов. Сделайте подписи сайтов вертикальными, иначе они сливаются (*xticks*).**

In [25]:
data = [go.Bar(
            x=top10_sites,
            y=top10_freqs
    )]

layout = go.Layout(title = 'Частоты посещений топ-10 сайтов', margin = {'b': 150})
fig = go.Figure(data = data, layout = layout)
iplot(fig, show_link = False)

## Часть 3. Дальнейшее построение признаков

Это задание творческое, тут надо придумать, как еще учесть время посещения веб-страниц и прочие признаки. 

На следующей неделе мы будем использовать "мешок" сайтов  для классификации сессий по принадлежности разным пользователям, а эти новые признаки, которые Вы сейчас создадите, потом добавим и посмотрим, улучшается ли модель. Поэтому можно их создать в виде отдельных матриц и сохранить их также отдельно.  

В этой части задания Вы можете построить и визуально исследовать самые разные признаки (ничто фантазию не ограничивает):
- время суток
- среднее время пребывания на сайте, посчитать можно, скажем, для топ-30 популярных сайтов
- индикаторы посещения популярных сайтов (скажем, тоже для топ-30 популярных сайтов)
- частота посещения Facebook
- ...

**Напишите функцию для создания новых признаков и примените ее к исходным данным – каталогам с 10 и 150 файлами. Сделайте это только для набора данных, полученного с параметрами *session_length=10* и *window_size=10*. Сериализуйте полученные матрицы с помощью pickle. Функция может возвращать как только новые признаки, так и старые с новыми. При этом сигнатура функции может быть другой – тут уже свобода выбора.**

Посчитаем число посещений в сессии одного из топ-30 сайтов, а также месяц и день месяца.

In [46]:
def construct_item(sites, user_id, session_length, timestamps, top_sites_ids):
    if len(sites) == 0:
        return None
    
    item = {'target': user_id}
    
    if len(sites) != session_length:
        sites = sites + [0]*(session_length - len(sites))
        timestamps = timestamps + [timestamps[-1]]*(session_length - len(timestamps))
    for i in range(session_length):
        item['site' + str(i+1)] = sites[i]
        if i < session_length - 1:
            item['time_diff' + str(i+1)] = timestamps[i+1] - timestamps[i]
    
    for i in range(len(top_sites_ids)):
        item['top_site' + str(i+1)] = len(filter(lambda x: x == top_sites_ids[i], sites))
    
    item['session_timespan'] = max(timestamps) - min(timestamps)
    item['#unique_sites'] = len(list(set(filter(lambda x: x != 0, sites))))
    item['start_hour'] = datetime.datetime.fromtimestamp(min(timestamps)).hour
    item['start_day'] = datetime.datetime.fromtimestamp(min(timestamps)).day
    item['start_month'] = datetime.datetime.fromtimestamp(min(timestamps)).month
    item['day_of_week'] = datetime.datetime.fromtimestamp(min(timestamps)).weekday()
    return item

def feature_engineering(csv_files_mask, site_freq_path, feature_names = None,
                                    session_length=10, window_size=10):
    pkl_file = open(site_freq_path, 'rb')
    site_freq = pickle.load(pkl_file)
    
    site_freq_df = pd.DataFrame.from_dict(site_freq).T
    site_freq_df.columns = ['site_id', 'site_freq']
    site_freq_df.sort_values('site_freq', ascending = False, inplace = True)
    top_sites_ids = site_freq_df.head(30).site_id.values.tolist()
    
    sessions = []
    
    for filename in glob(csv_files_mask):
        with open(filename) as input_file:
            data = filter(lambda x: x!= '', map(lambda x: x.strip(), input_file.readlines()))

            user_id = int(data[0].split(',')[0])
            n = len(data)
            for i in range(0, n, window_size):
                session = map(lambda x: x.split(',')[2], data[i:i + session_length])
                times = map(lambda x: x.split(',')[1], data[i:i + session_length])
                if len(session) == 0:
                    continue
                session_ids = map(lambda x: site_freq[x][0], session)
                timestamps = map(lambda x: int(datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S').strftime('%s')), 
                                 times)
                
                sessions.append(construct_item(session_ids, user_id, session_length, timestamps, top_sites_ids))
    
    df = pd.DataFrame(sessions)
    df = df.drop_duplicates()

    columns = map(lambda x: 'site' + str(x+1), range(session_length)) + ['target']
    if feature_names is not None:
        df = df[feature_names]
    return df

In [47]:
new_features_10users = feature_engineering('capstone_websites_data/10users/*', 
                                   site_freq_path='capstone_websites_data/site_freq_10users.pkl')

new_features_10users['target'] = new_features_10users['target'].map(id_name_dict)

In [48]:
new_features_150users = feature_engineering('capstone_websites_data/150users/*', 
                                   site_freq_path='capstone_websites_data/site_freq_150users.pkl') 

**10. Постройте картинки для новых признаков, поисследуйте их, прокомментируйте результаты.**

In [49]:
data = [go.Histogram(
    x = new_features_10users['start_day'] ,
    marker = {'color': 'dark_green'}
)]
layout = go.Layout(
    xaxis=dict(
        title='День месяца начала сессии'
    ),
    yaxis=dict(
        title='Число сессий'
    )
)

fig = go.Figure(data = data, layout = layout)
iplot(fig, show_link=False)

In [50]:
fig = tools.make_subplots(rows=5, cols=2)

for idx, (user, sub_df) in  enumerate(pd.groupby(new_features_10users, 'target')): 
    trace = go.Histogram(
        x = sub_df['start_day'] ,
        marker = {'color': color_dic[user]},
        name = user
    )
    fig.append_trace(trace, int(idx/2) + 1, int(idx % 2) + 1)
    fig['layout']['xaxis' + str(idx + 1)].update(title = 'День начала сессии')
    fig['layout']['yaxis' + str(idx + 1)].update(title = 'Число сессий')
    
fig['layout'].update(height=1100, width=800)
iplot(fig, show_link = False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]  
[ (2,1) x3,y3 ]    [ (2,2) x4,y4 ]  
[ (3,1) x5,y5 ]    [ (3,2) x6,y6 ]  
[ (4,1) x7,y7 ]    [ (4,2) x8,y8 ]  
[ (5,1) x9,y9 ]    [ (5,2) x10,y10 ]



In [51]:
data = [go.Histogram(
    x = new_features_10users['start_month'] ,
    marker = {'color': 'dark_green'}
)]
layout = go.Layout(
    xaxis=dict(
        title='Месяц начала сессии'
    ),
    yaxis=dict(
        title='Число сессий'
    )
)

fig = go.Figure(data = data, layout = layout)
iplot(fig, show_link=False)

In [52]:
fig = tools.make_subplots(rows=5, cols=2)

for idx, (user, sub_df) in  enumerate(pd.groupby(new_features_10users, 'target')): 
    trace = go.Histogram(
        x = sub_df['start_month'] ,
        marker = {'color': color_dic[user]},
        name = user
    )
    fig.append_trace(trace, int(idx/2) + 1, int(idx % 2) + 1)
    fig['layout']['xaxis' + str(idx + 1)].update(title = 'Месяц начала сессии')
    fig['layout']['yaxis' + str(idx + 1)].update(title = 'Число сессий')
    
fig['layout'].update(height=1100, width=800)
iplot(fig, show_link = False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]  
[ (2,1) x3,y3 ]    [ (2,2) x4,y4 ]  
[ (3,1) x5,y5 ]    [ (3,2) x6,y6 ]  
[ (4,1) x7,y7 ]    [ (4,2) x8,y8 ]  
[ (5,1) x9,y9 ]    [ (5,2) x10,y10 ]



У пользователей наблюдается различная активность в зависимости от дня, по месяцам также видны различия.

**В конце сохраните в pickle-файлы только те признаки, которые, как Вы предполагаете, помогут идентифицировать пользователя более точно. Это касается и признаков, которые мы вместе создали в начале (*session_timespan, time_diff1,..., time_diff9, #unique_sites, start_hour, day_of_week*), и Ваших собственных. Можно создать все эти признаки не только для сессий из 10 сайтов, но и для других сочетаний параметров *session_length* и *window_size*.**

In [41]:
selected_features = [u'#unique_sites', u'day_of_week', u'session_timespan', u'site1',
       u'site10', u'site2', u'site3', u'site4', u'site5', u'site6', u'site7',
       u'site8', u'site9', u'start_hour', u'target', u'time_diff1',
       u'time_diff2', u'time_diff3', u'time_diff4', u'time_diff5',
       u'time_diff6', u'time_diff7', u'time_diff8', u'time_diff9',
       u'top_site1', u'top_site10', u'top_site11', u'top_site12',
       u'top_site13', u'top_site14', u'top_site15', u'top_site16',
       u'top_site17', u'top_site18', u'top_site19', u'top_site2',
       u'top_site20', u'top_site21', u'top_site22', u'top_site23',
       u'top_site24', u'top_site25', u'top_site26', u'top_site27',
       u'top_site28', u'top_site29', u'top_site3', u'top_site30', u'top_site4',
       u'top_site5', u'top_site6', u'top_site7', u'top_site8', u'top_site9']

In [42]:
selected_features_10users = new_features_10users[selected_features]
selected_features_150users = new_features_150users[selected_features]

In [43]:
# selected features – among (session_timespan, ..., day_of_week) and your own new ones
with open('selected_features_10users.pkl', 'wb') as selected_features_10users_pkl:
    pickle.dump(selected_features_10users, selected_features_10users_pkl)
with open('selected_features_150users.pkl', 'wb') as selected_features_150users_pkl:
    pickle.dump(selected_features_150users, selected_features_150users_pkl)

## Критерии оценки работы:
- Верно ли отображена гистограмма session_timespan из п. 1? (max. 3 балла)
- Верно ли отображена гистограмма #unique_sites из п. 2? (max. 3 балла)
- Верно ли отображены гистограммы #unique_sites по каждому пользователю из п. 3? (max. 6 баллов)
- Верно ли отображена гистограмма start_hour из п. 4? (max. 3 балла)
- Верно ли отображены гистограммы start_hour по каждому пользователю из п. 5? (max. 6 баллов)
- Верно ли отображена гистограмма day_of_week из п. 6? (max. 3 балла)
- Верно ли отображены гистограммы day_of_week по каждому пользователю из п. 7? (max. 6 баллов)
- Насколько сделанные выводы в п. 8 соответствуют построенным картинкам? (max. 6 баллов)
- Верно ли отображен barplot для 10 популярных сайтов из п. 9? (max. 6 баллов)
- Есть ли оригинальные построенные признаки и картинки к ним? Оцените также и качество картинок. (max. 8 баллов)

## Пути улучшения
7 неделя проекта посвящена общему описанию проекта (.ipynb или pdf) и взаимному оцениванию. Что еще можно добавить по 3 части проекта:
- IPython-widgets, интерактив и анимация (стоящая [статья](https://habrahabr.ru/post/308162/) по этому ремеслу)
- можно попробовать изобразить исходные данные в некотором пространстве, например, Word2Vec, потом выделить главные компоненты или t-SNE  (только пользуйтесь эффективными реализациями, не Sklearn) и раскрасить по целевому классу. Но нет гарантий, что получится что-то значимо отличающееся от каши

Далее пройдите чисто техническое задание "Проверка построенных признаков", цель которого – удостовериться, что все мы одинаково создали признаки *session_timespan, time_diff1,..., time_diff9, #unique_sites, start_hour* и *day_of_week*.